In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor

In [2]:
# Load the dataset
data = pd.read_csv(r'C:\Users\Vijay\Downloads\financial_anomaly_data.csv')

In [3]:
data.dropna(inplace=True)

In [4]:
# Select relevant columns for anomaly detection
columns_for_detection = ['Amount', 'Merchant', 'TransactionType', 'Location']
selected_data = data[columns_for_detection]

In [5]:
# Perform label encoding for categorical columns
label_encoders = {}
for col in selected_data.select_dtypes(include='object').columns:
    label_encoders[col] = LabelEncoder()
    selected_data[col] = label_encoders[col].fit_transform(selected_data[col])

C:\Users\Vijay\AppData\Local\Temp\ipykernel_9468\1120016891.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data[col] = label_encoders[col].fit_transform(selected_data[col])
C:\Users\Vijay\AppData\Local\Temp\ipykernel_9468\1120016891.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data[col] = label_encoders[col].fit_transform(selected_data[col])
C:\Users\Vijay\AppData\Local\Temp\ipykernel_9468\1120016891.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [6]:
# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(selected_data)

In [7]:
# K-means clustering
kmeans = KMeans(n_clusters=2)  # Assuming 2 clusters
data['kmeans_cluster'] = kmeans.fit_predict(scaled_data)

C:\Users\Vijay\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [8]:
# Isolation Forest
isolation_forest = IsolationForest(contamination=0.01)  # Adjust contamination as needed
data['isolation_forest_anomaly'] = isolation_forest.fit_predict(scaled_data)

In [9]:
# One-Class SVM
svm = OneClassSVM(nu=0.01)  # Adjust nu parameter as needed
data['svm_anomaly'] = svm.fit_predict(scaled_data)

In [10]:
# Local Outlier Factor
lof = LocalOutlierFactor(novelty=False)
data['lof_anomaly'] = lof.fit_predict(scaled_data)

In [11]:
# Assign labels to detected anomalies
data['anomaly_label'] = 'Normal'
data.loc[data['isolation_forest_anomaly'] == -1, 'anomaly_label'] = 'Anomaly_IF'
data.loc[data['svm_anomaly'] == -1, 'anomaly_label'] = 'Anomaly_SVM'
data.loc[data['lof_anomaly'] == -1, 'anomaly_label'] = 'Anomaly_LOF'

In [12]:
data

,Timestamp,TransactionID,AccountID,Amount,Merchant,TransactionType,Location,kmeans_cluster,isolation_forest_anomaly,svm_anomaly,lof_anomaly,anomaly_label
0,01-01-2023 08:00,TXN1127,ACC4,95071.92,MerchantH,Purchase,Tokyo,1,1,1,1,Normal
1,01-01-2023 08:01,TXN1639,ACC10,15607.89,MerchantH,Purchase,London,1,1,1,1,Normal
2,01-01-2023 08:02,TXN872,ACC8,65092.34,MerchantE,Withdrawal,London,0,1,1,1,Normal
3,01-01-2023 08:03,TXN1438,ACC6,87.87,MerchantE,Purchase,London,0,1,-1,1,Anomaly_SVM
4,01-01-2023 08:04,TXN1338,ACC6,716.56,MerchantI,Purchase,Los Angeles,1,1,1,1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...
216955,31-05-2023 23:55,TXN1286,ACC6,62536.88,MerchantA,Withdrawal,San Francisco,0,1,1,1,Normal
216956,31-05-2023 23:56,TXN1015,ACC5,68629.69,MerchantG,Transfer,London,1,1,1,1,Normal
216957,31-05-2023 23:57,TXN1979,ACC15,8203.57,MerchantF,Purchase,London,1,1,1,1,Normal
216958,31-05-2023 23:58,TXN1845,ACC14,77800.36,MerchantF,Purchase,New York,1,1,1,1,Normal


In [13]:
# Sum of anomalies predicted by each algorithm
data['total_anomalies'] = data[['isolation_forest_anomaly', 'svm_anomaly', 'lof_anomaly']].sum(axis=1)

# Label a transaction as an anomaly if at least 2 out of 3 algorithms predict it as an anomaly
data['anomaly_label'] = 'Normal'
data.loc[data['total_anomalies'] <= -1, 'anomaly_label'] = 'Anomaly'

# Drop intermediate columns used for calculations if needed
data.drop(['isolation_forest_anomaly', 'svm_anomaly', 'lof_anomaly', 'total_anomalies'], axis=1, inplace=True)


In [14]:
data

,Timestamp,TransactionID,AccountID,Amount,Merchant,TransactionType,Location,kmeans_cluster,anomaly_label
0,01-01-2023 08:00,TXN1127,ACC4,95071.92,MerchantH,Purchase,Tokyo,1,Normal
1,01-01-2023 08:01,TXN1639,ACC10,15607.89,MerchantH,Purchase,London,1,Normal
2,01-01-2023 08:02,TXN872,ACC8,65092.34,MerchantE,Withdrawal,London,0,Normal
3,01-01-2023 08:03,TXN1438,ACC6,87.87,MerchantE,Purchase,London,0,Normal
4,01-01-2023 08:04,TXN1338,ACC6,716.56,MerchantI,Purchase,Los Angeles,1,Normal
...,...,...,...,...,...,...,...,...,...
216955,31-05-2023 23:55,TXN1286,ACC6,62536.88,MerchantA,Withdrawal,San Francisco,0,Normal
216956,31-05-2023 23:56,TXN1015,ACC5,68629.69,MerchantG,Transfer,London,1,Normal
216957,31-05-2023 23:57,TXN1979,ACC15,8203.57,MerchantF,Purchase,London,1,Normal
216958,31-05-2023 23:58,TXN1845,ACC14,77800.36,MerchantF,Purchase,New York,1,Normal


In [19]:
Anamoly = data[data['anomaly_label']=='Anomaly']

In [21]:
Anamoly['TransactionType'].value_counts()

Purchase      829
Withdrawal    670
Transfer        5
Name: TransactionType, dtype: int64

In [22]:
data['TransactionType'].value_counts()

Transfer      72793
Purchase      72235
Withdrawal    71932
Name: TransactionType, dtype: int64

In [26]:
Anamoly['Amount'].agg(['min','max','mean'])

min         11.160000
max     978942.260000
mean     57106.166316
Name: Amount, dtype: float64